# Visual Auditor Package Demo (Customer Churn Prediction Dataset)
Author: David Munechika (david.munechika@gatech.edu)

### Install Packages and Libraries

In [1]:
# Import packages
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from visual_auditor import SliceFinder
import json

# Import visual auditor module
import visual_auditor

# Expand Jupyter notebook width
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

### Data Preprocessing and Model Training

In [12]:
# Helper function for binning numerical features
def bin_feature(feature):
    bins = np.histogram_bin_edges(customer_churn[feature], bins=10, range=None, weights=None)
    customer_churn[feature] = pd.cut(customer_churn[feature], bins, labels=[x for x in range(len(bins) - 1)], right=True, include_lowest=True, duplicates='drop')
    intervals = []
    for i in range(len(bins) - 1):
        intervals.append(f' {int(bins[i])} - {int(bins[i+1])}')
    return intervals

# Helper function for mapping encodings to original values
def map_encodings(filename, features, encodings):
    slices_file = open(filename, "r")
    data = json.dumps(json.loads(slices_file.read()))
    slices_file.close()
    for feature in features:
        for i in range(len(encodings[feature])):
            index = len(encodings[feature]) - i - 1
            data = data.replace(f'{feature}: {index}', f'{feature}: {encodings[feature][index]}')
    with open(filename, 'w') as outfile:
        json.dump(json.JSONDecoder().decode(data), outfile)

In [13]:
# Load Customer Churn Prediction dataset
customer_churn = pd.read_csv(
    "data/customer_churn_prediction.csv",
    names=[
        "State", "Account Length", "Area Code", "International", "Voicemail Plan", "Voicemail Messages",
        "Day Minutes", "Day Calls", "Day Charge", "Eve Minutes", "Eve Calls", "Eve Charge", "Night Minutes",
        "Night Calls", "Night Charge", "Intl Minutes", "Intl Calls", "Intl Charge", "CustServ Calls", "Churn"],
        engine='python',
        na_values="NA")

# Drop NA values
customer_churn = customer_churn.dropna()

# Bin numerical features
encoders = {}
encodings = {}
numerical_features = ["Account Length", "Voicemail Messages", "Day Minutes", "Eve Minutes", "Night Minutes", "Intl Minutes", "Day Charge",
                     "Eve Charge", "Night Charge", "Intl Charge", "Day Calls", "Eve Calls", "Night Calls", "Intl Calls", "CustServ Calls"]
for feature in numerical_features:
    encodings[feature] = bin_feature(feature)

# Encode categorical features
for column in customer_churn.columns.difference(numerical_features):
    if customer_churn.dtypes[column] == np.object:
        le = LabelEncoder()
        customer_churn[column] = le.fit_transform(customer_churn[column])
        encoders[column] = le
        encodings[column] = le.classes_
        print(column, le.classes_, le.transform(le.classes_))

# Separate Target values
X, y = customer_churn[customer_churn.columns.difference(["Churn"])], customer_churn["Churn"]

# Train a classifier
rfc = RandomForestClassifier(max_depth=5, n_estimators=10)
rfc.fit(X, y)

Area Code ['area_code_408' 'area_code_415' 'area_code_510'] [0 1 2]
Churn ['no' 'yes'] [0 1]
International ['no' 'yes'] [0 1]
State ['AK' 'AL' 'AR' 'AZ' 'CA' 'CO' 'CT' 'DC' 'DE' 'FL' 'GA' 'HI' 'IA' 'ID'
 'IL' 'IN' 'KS' 'KY' 'LA' 'MA' 'MD' 'ME' 'MI' 'MN' 'MO' 'MS' 'MT' 'NC'
 'ND' 'NE' 'NH' 'NJ' 'NM' 'NV' 'NY' 'OH' 'OK' 'OR' 'PA' 'RI' 'SC' 'SD'
 'TN' 'TX' 'UT' 'VA' 'VT' 'WA' 'WI' 'WV' 'WY'] [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50]
Voicemail Plan ['no' 'yes'] [0 1]
{'Account Length': [' 1 - 25', ' 25 - 49', ' 49 - 73', ' 73 - 97', ' 97 - 122', ' 122 - 146', ' 146 - 170', ' 170 - 194', ' 194 - 218', ' 218 - 243'], 'Voicemail Messages': [' 0 - 5', ' 5 - 10', ' 10 - 15', ' 15 - 20', ' 20 - 26', ' 26 - 31', ' 31 - 36', ' 36 - 41', ' 41 - 46', ' 46 - 52'], 'Day Minutes': [' 0 - 35', ' 35 - 70', ' 70 - 105', ' 105 - 140', ' 140 - 175', ' 175 - 210', ' 210 - 246', ' 246 - 281', ' 2

### Interact with Visual Auditor

In [19]:
visual_auditor.find_slices_and_visualize(rfc, (X, y), k=100, epsilon=0.2, degree=2, max_workers=4, precompute=True, prefix='cp_')

<iframe
 srcdoc="<!doctype html><html lang="en"><head><meta charset="utf-8"/><link rel="icon" href="/favicon.ico"/><meta name="viewport" content="width=device-width,initial-scale=1"/><meta name="theme-color" content="#000000"/><meta name="description" content="Interactive Scalable Auditing of Model Biases and Vulnerabilities with Interpretable Mitigation"/><link rel="apple-touch-icon" href="/logo192.png"/><link rel="manifest" href="/manifest.json"/><title>Visual Auditor</title><style>body{margin:0;font-family:-apple-system,BlinkMacSystemFont,"Segoe UI",Roboto,Oxygen,Ubuntu,Cantarell,"Fira Sans","Droid Sans","Helvetica Neue",sans-serif;-webkit-font-smoothing:antialiased;-moz-osx-font-smoothing:grayscale;background:#fff}code{font-family:source-code-pro,Menlo,Monaco,Consolas,"Courier New",monospace}*{scrollbar-width:thin;scrollbar-color:hsla(0,0%,60.8%,.5) transparent}::-webkit-scrollbar{width:5px}::-webkit-scrollbar-track{background:0 0}::-webkit-scrollbar-thumb{background-color:hsla(0,0%,60.8%,.5);border-radius:20px;border:transparent}.nav-container{text-align:center;z-index:2}.settings{margin:1rem}.settings:hover{opacity:1%;cursor:pointer}.left-container{text-align:center;padding:.5rem}.MuiFormControlLabel-label{color:#5f5f5f;font-weight:700}h1{color:#5f5f5f}h2{font-size:1.25rem;padding:.5rem}h2,p{color:#5f5f5f;font-weight:300}p{text-align:left;font-size:1.1rem;padding:0 1rem}.similar-slices-list{font-size:1rem;color:#5f5f5f;font-weight:300;text-align:left}.similar-slice{padding:.25rem 0}.thin{font-weight:300}.right-container{text-align:center;padding:.5rem}.main-container{padding:1rem;text-align:left;overflow-x:scroll;background-color:#fff}img{margin-left:10rem;margin-right:10rem;width:60rem}.hull{fill:rgba(184,125,0,.705);stroke:rgba(184,125,0,.705);opacity:.5;stroke-width:32px;stroke-linejoin:round}.link{stroke:#000;stroke-width:1.5px}.node{cursor:move;fill:#ccc;stroke-width:1.5px}.node.fixed{fill:red}</style></head><body><noscript>You need to enable JavaScript to run this app.</noscript><div id="root"></div><script>!function(e){function r(r){for(var n,i,l=r[0],a=r[1],f=r[2],c=0,s=[];c<l.length;c++)i=l[c],Object.prototype.hasOwnProperty.call(o,i)&&o[i]&&s.push(o[i][0]),o[i]=0;for(n in a)Object.prototype.hasOwnProperty.call(a,n)&&(e[n]=a[n]);for(p&&p(r);s.length;)s.shift()();return u.push.apply(u,f||[]),t()}function t(){for(var e,r=0;r<u.length;r++){for(var t=u[r],n=!0,l=1;l<t.length;l++){var a=t[l];0!==o[a]&&(n=!1)}n&&(u.splice(r--,1),e=i(i.s=t[0]))}return e}var n={},o={1:0},u=[];function i(r){if(n[r])return n[r].exports;var t=n[r]={i:r,l:!1,exports:{}};return e[r].call(t.exports,t,t.exports,i),t.l=!0,t.exports}i.m=e,i.c=n,i.d=function(e,r,t){i.o(e,r)||Object.defineProperty(e,r,{enumerable:!0,get:t})},i.r=function(e){"undefined"!=typeof Symbol&&Symbol.toStringTag&&Object.defineProperty(e,Symbol.toStringTag,{value:"Module"}),Object.defineProperty(e,"__esModule",{value:!0})},i.t=function(e,r){if(1&r&&(e=i(e)),8&r)return e;if(4&r&&"object"==typeof e&&e&&e.__esModule)return e;var t=Object.create(null);if(i.r(t),Object.defineProperty(t,"default",{enumerable:!0,value:e}),2&r&&"string"!=typeof e)for(var n in e)i.d(t,n,function(r){return e[r]}.bind(null,n));return t},i.n=function(e){var r=e&&e.__esModule?function(){return e.default}:function(){return e};return i.d(r,"a",r),r},i.o=function(e,r){return Object.prototype.hasOwnProperty.call(e,r)},i.p="/";var l=this["webpackJsonpvisual-auditor"]=this["webpackJsonpvisual-auditor"]||[],a=l.push.bind(l);l.push=r,l=l.slice();for(var f=0;f<l.length;f++)r(l[f]);var p=a;t()}([])</script><script type="text/javascript">/*! For license information please see 2.d769a9af.chunk.js.LICENSE.txt */
(this["webpackJsonpvisual-auditor"]=this["webpackJsonpvisual-auditor"]||[]).push([[2],[function(e,t,n){"use strict";e.exports=n(234)},function(e,t,n){"use strict";e.exports=n(240)},function(e,t,n){"use strict";function r(){return(r=Object.assign||function(e){for(var t=1;t<arguments.length;t++){var n=argument

### Precompute Data for Demo (Internal Use)

In [18]:
# Map encodings
features = ["State", "Account Length", "Area Code", "International", "Voicemail Plan", "Voicemail Messages",
        "Day Minutes", "Day Calls", "Day Charge", "Eve Minutes", "Eve Calls", "Eve Charge", "Night Minutes",
        "Night Calls", "Night Charge", "Intl Minutes", "Intl Calls", "Intl Charge", "CustServ Calls"]
map_encodings('cp_slices.json', features, encodings)
map_encodings('cp_common_samples.json', features, encodings)
map_encodings('cp_overlapping_samples.json', features, encodings)
map_encodings('cp_reverse_slices.json', features, encodings)
map_encodings('cp_reverse_common_samples.json', features, encodings)
map_encodings('cp_reverse_overlapping_samples.json', features, encodings)